# Grouping Companies for non-downtown buildings

In [1]:
from utils import owners
from utils import geo

In [2]:
import pandas as pd
import numpy as np
import requests
import json
import os
import re
import geopandas as gp
import urllib.parse

## Step 1: Map owner names to entries in the CCFS database

In [46]:
df_districts = gp.read_file("../../../../data/Council_Districts.geojson")
df = pd.read_csv('../../../../data/2020_Building_Energy_Benchmarking.csv')
df = gp.GeoDataFrame(df, geometry=gp.points_from_xy(df.Longitude, df.Latitude))
geo.clean_districts(df, df_districts)

df_filtered = df.loc[df['Neighborhood'] == "DOWNTOWN"]

building_owners = pd.read_csv('../../../../experiments/worst_offenders/updated_owners_2_15_23.csv')
# Map tax ids to landlord name
d = pd.Series(building_owners.Owner.values, index=building_owners.TaxParcelIdentificationNumber).to_dict()
df_filtered['Landlord'] = df_filtered['TaxParcelIdentificationNumber'].map(lambda row: d.get(str(row), ""))

df_filtered.head()

Building WATERWORKS OFFICE & MARINA 2353/ 4088803975 doesn't have a district POINT (-122.33895 47.63575) 
	 Found district 7 for WATERWORKS OFFICE & MARINA
Building NAUTICAL LANDING 2381/ 4088804350 doesn't have a district POINT (-122.34219 47.64306) 
	 Found district 7 for NAUTICAL LANDING
Building UNION HARBOR CONDOMINIUM 2540/ 8807200000 doesn't have a district POINT (-122.33003 47.6401) 
	 Found district 4 for UNION HARBOR CONDOMINIUM
Building THE PIER AT LESCHI 2997/ 6780900000 doesn't have a district POINT (-122.28563 47.59926) 
	 Found district 3 for THE PIER AT LESCHI
Building THE LAKESHORE 3046/ 1180001715 doesn't have a district POINT EMPTY 
Building EDUCARE 3218/ 2895800030 doesn't have a district POINT EMPTY 


c:\Users\linne\miniconda3\envs\bps\lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,OSEBuildingID,DataYear,BuildingName,BuildingType,TaxParcelIdentificationNumber,Address,City,State,ZipCode,Latitude,...,SteamUse(kBtu),NaturalGas(therms),ComplianceStatus,ComplianceIssue,Electricity(kBtu),NaturalGas(kBtu),TotalGHGEmissions,GHGEmissionsIntensity,geometry,Landlord
0,1,2020,MAYFLOWER PARK HOTEL,NonResidential,0659000030,405 OLIVE WAY,SEATTLE,WA,98101.0,47.61220,...,1457837,6326,Compliant,No Issue,2734351.0,632586.0,169.1,1.9,POINT (-122.33799 47.61220),
1,2,2020,PARAMOUNT HOTEL,NonResidential,0659000220,724 PINE ST,SEATTLE,WA,98101.0,47.61317,...,0,16614,Compliant,No Issue,1940292.0,1661402.0,98.6,1.1,POINT (-122.33393 47.61317),
2,3,2020,WESTIN HOTEL (Parent Building),NonResidential,0659000475,1900 5TH AVE,SEATTLE,WA,98101.0,47.61367,...,10359896,8955,Compliant,No Issue,25517379.0,895500.0,1043.2,1.4,POINT (-122.33822 47.61367),
3,5,2020,HOTEL MAX,NonResidential,0659000640,620 STEWART ST,SEATTLE,WA,98101.0,47.61412,...,917724,8871,Compliant,No Issue,1177927.0,887059.0,129.6,2.1,POINT (-122.33664 47.61412),
4,8,2020,WARWICK SEATTLE HOTEL,NonResidential,0659000970,401 LENORA ST,SEATTLE,WA,98121.0,47.61375,...,0,46034,Compliant,No Issue,3761566.0,4603411.0,264.5,2.3,POINT (-122.34047 47.61375),


In [37]:
building_owners[building_owners["TaxParcelIdentificationNumber"] == "8592900105"]

,Unnamed: 0,OSEBuildingID,TaxParcelIdentificationNumber,DataYear,BuildingType,BuildingName,Owner,CouncilDistrictCode,Neighborhood,Units,...,SecondLargestPropertyUseTypeGFA,ThirdLargestPropertyUseType,ThirdLargestPropertyUseTypeGFA,Outlier,ComplianceIssue,ComplianceStatus,Comments,DefaultData,LegislationPropertyType,Dept
3228,3228,278,8592900105,2020,NonResidential,ID VILLAGE SQUARE 1,700 OLIVE LP % CLISE PROPERTIES,1.0,DOWNTOWN,NOT_FOUND,...,40091.0,K-12 School,3231.0,NaN,No Issue,Compliant,NaN,NaN,Other,NaN


In [56]:
df_filtered = df.loc[df['Neighborhood'] == "DOWNTOWN"]
df_90k = df_filtered[df_filtered["PropertyGFABuilding(s)"] > 90000]
df_90k = df_90k[df_90k['PropertyGFABuilding(s)'] < 100000]

In [57]:
df_filtered = df_90k

In [66]:
df_filtered['SearchTerm'] = df_filtered['TaxParcelIdentificationNumber'].map(lambda row: d.get(str(row), ""))

In [67]:
unique_not_downtown_landlords = df_filtered['Landlord'].unique()
unique_not_downtown_landlords = pd.DataFrame(unique_not_downtown_landlords, columns=['owner_name'])
unique_not_downtown_landlords = unique_not_downtown_landlords[~unique_not_downtown_landlords['owner_name'].isin(['NOT FOUND', 'UNDEFINED'])]
unique_not_downtown_landlords.to_csv('unique_90k_downtown.csv')
owner_search_list = list(unique_not_downtown_landlords['owner_name'])

In [68]:
owner_search_list

['700 OLIVE LP  % CLISE PROPERTIES',
 '',
 'WESTERN OFFICE PORTFOLIO PROPERTY OWNER LLC',
 'PUBLIC STORAGE',
 'FG92 LENORA LLC+CWS LENORA LP',
 'PLYMOUTH HOUSING GROUP',
 'PIONEER PRESERVATION ASSOCIATES LP',
 'DA LI INTERNATIONAL LLC']

In [70]:
lookup_helper = owners.LookupCompaniesHelper(os.getcwd())
lookup_helper.get_company_matches_and_export(owner_search_list[:10],1)

Saving output files to c:\Users\linne\Documents\BPS\experiments\landlords\parent_company_search\non-downtown buildings


KeyError: 'SearchTerm'

## Step 2: Get all companies and their principals
Now that we've mapped company names to their entries in the CCFS database, we create a list of all the companies and all the principals registered to that company. This all-matches-all-principals will be used in step 3 to iterate through and group by shared principals. 

In [ ]:
group_helper = owners.GroupCompaniesHelper(os.getcwd(), "companies_and_potential_matches_ntd.csv")

In [ ]:
exact_matches_1 = pd.read_csv("exact_matches_1.csv")
exact_matches_1_principals = group_helper.get_companies_principals(exact_matches_1)

In [ ]:
all_matches = pd.DataFrame([])
for i in range(1, 11):
    print(f"Getting principals for exact_matches_{i}")
    exact_matches = pd.read_csv(f"exact_matches_{i}.csv")
    exact_matches_principals = group_helper.get_companies_principals(exact_matches)
    all_matches = pd.conact([all_matches, exact_matches_principals])

for i in range(1, 10):
    print(f"Getting principals for potential_matches_{i}")
    potential_matches = pd.read_csv(f"potential_matches_{i}.csv")
    potential_matches = potential_matches[potential_matches['isMatch']==1]
    potential_matches_principals = group_helper.get_companies_principals(potential_matches)
    all_matches = pd.conact([all_matches, potential_matches_principals])

# Step 3: Group Companies by shared principals
Now that we have all of the companies in CCFS database and all of the principals registered to that company, we can group all of the results by shared principals. 

In [ ]:
companies_and_matches = group_helper.group_companies_by_principals(all_matches)